<a class="anchor" id="0"></a>
# [Mechanisms of Action (MoA) Prediction](https://www.kaggle.com/c/lish-moa)

# **This notebook Introduces transfer learning with sampling techniques .**


<h1 style="color:green"> PLEASE CONSIDER UPVOTING THANKS!!!!!! </h1>

<br>the model is first trained on non scored targets and then that is used to train the scored targets.



Base for this notebook - 
https://www.kaggle.com/chriscc/kubi-pytorch-moa-transfer<br>
https://www.kaggle.com/ash1706/moa-randomoversampler-drug-id-fixed-imbalance



Please use the version 3 of this notebook for sampling techniques along with tranfer learning or uncomment the sampling part here.

Assumptions:
* MoA could be reduced to multiclass using drug_id making X = features and y= drug_id
* Assuming drug_id has a pattern and relation with targets 


In [ ]:
import sys
sys.path.append('../input/iterativestratification')


import numpy as np
import random
import pandas as pd
import os
import copy
import gc


import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA , TruncatedSVD
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import scipy.stats as stats
from scipy.stats import kurtosis

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.modules.loss import _WeightedLoss

import warnings
warnings.filterwarnings('ignore')

os.listdir('../input/lish-moa')

pd.set_option('max_columns', 2000)

In [ ]:

n_comp_GENES = 450 
n_comp_CELLS = 2
VarianceThreshold_for_FS = 0.9
Dropout_Model = 0.25
QT_n_quantile_min=50, 
QT_n_quantile_max=1000,
print('n_comp_GENES', n_comp_GENES, 'n_comp_CELLS', n_comp_CELLS, 'total', n_comp_GENES + n_comp_CELLS)

In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
train_drugs = pd.read_csv('../input/lish-moa/train_drug.csv')


test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

## 4. FE & Data Preprocessing 



In [ ]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

### 4.1 RankGauss<a class="anchor" id="4.1"></a>

[Back to Table of Contents](#0.1)

In [ ]:
# Search for minimum and maximum values
# df_kurt = pd.DataFrame(columns=['col','train', 'test'])
# i = 0
# for col in (GENES + CELLS):
#     df_kurt.loc[i, 'col'] = col
#     df_kurt.loc[i, 'train'] = kurtosis(train_features[col])
#     df_kurt.loc[i, 'test'] = kurtosis(test_features[col])
#     i += 1
# print(df_kurt.min())
# print(df_kurt.max())

In [ ]:
def calc_QT_par_kurt(QT_n_quantile_min=10, QT_n_quantile_max=200):
    # Calculation parameters of function: n_quantile(kurtosis) = k1*kurtosis + k0
    # For Train & Test datasets (GENES + CELLS features): minimum kurtosis = 1.53655, maximum kurtosis = 30.4929
    
    a = np.array([[1.53655,1], [30.4929,1]])
    b = np.array([QT_n_quantile_min, QT_n_quantile_max])
    
    return np.linalg.solve(a, b)

In [ ]:
def n_quantile_for_kurt(kurt, calc_QT_par_kurt_transform):
    # Calculation parameters of function: n_quantile(kurtosis) = calc_QT_par_kurt_transform[0]*kurtosis + calc_QT_par_kurt_transform[1]
    return int(calc_QT_par_kurt_transform[0]*kurt + calc_QT_par_kurt_transform[1])

In [ ]:
# RankGauss - transform to Gauss

for col in (GENES + CELLS):

    #kurt = max(kurtosis(train_features[col]), kurtosis(test_features[col]))
    #QuantileTransformer_n_quantiles = n_quantile_for_kurt(kurt, calc_QT_par_kurt(QT_n_quantile_min, QT_n_quantile_max))
    #transformer = QuantileTransformer(n_quantiles=QuantileTransformer_n_quantiles,random_state=0, output_distribution="normal")
    
    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")   # from optimal commit 9
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

### 4.2 Seed


In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

### 4.3 PCA features



In [ ]:
len(GENES)

In [ ]:
# GENES

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (TruncatedSVD(n_components=n_comp_GENES, random_state=42).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp_GENES)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp_GENES)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [ ]:
len(CELLS)

In [ ]:
# CELLS

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (TruncatedSVD(n_components=n_comp_CELLS, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp_CELLS)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp_CELLS)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [ ]:
train_features.shape

In [ ]:
train_features.head(5)

In [ ]:
train_features = train_features.drop(CELLS+GENES , axis=1)

In [ ]:
test_features  = test_features.drop(CELLS+GENES , axis=1)

### 4.4 Feature selection



In [ ]:
data = train_features.append(test_features)
data

In [ ]:
var_thresh = VarianceThreshold(VarianceThreshold_for_FS)
data = train_features.append(test_features)
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]


train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features.shape

In [ ]:
train_features.head(5)

In [ ]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target = train[train_targets_scored.columns]

In [ ]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [ ]:
train.head(5)

In [ ]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

In [ ]:
target_non_scored_cols = train_targets_nonscored.drop('sig_id', axis=1).columns.values.tolist()

In [ ]:
train_non_scored =  train.drop(target_cols , axis = 1).copy()

In [ ]:
train_non_scored = train_non_scored.merge(train_targets_nonscored , how = 'inner')

In [ ]:
target_non_scored = train_non_scored[['sig_id']+target_non_scored_cols]

In [ ]:
train_non_scored

In [ ]:
train

### 4.5 CV folds



In [ ]:
folds_non_scored = train_non_scored.copy()

mskf = MultilabelStratifiedKFold(n_splits=7)

for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
    folds_non_scored.loc[v_idx, 'kfold'] = int(f)

folds_non_scored['kfold'] = folds_non_scored['kfold'].astype(int)
folds_non_scored

In [ ]:
folds = train.copy()

mskf = MultilabelStratifiedKFold(n_splits=7)

for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
    folds.loc[v_idx, 'kfold'] = int(f)

folds['kfold'] = folds['kfold'].astype(int)
folds

In [ ]:
print(train.shape)
print(folds.shape)
print(test.shape)
print(target.shape)
print(sample_submission.shape)

In [ ]:
folds

In [ ]:
train_drugs.head()

# Sampling 

uncomment the below lines

In [ ]:
# max(train_drugs['sig_id'].value_counts().values)  
# only one sig_id hence no multilabel buisness here ie no sig_id has two drugs_id 
# this is the base of turning the problem in multiclass for sampling purposes

In [ ]:
# # original train data set
# target_f =  train[target_cols]
# top_targets = pd.Series(target_f.sum()).sort_values(ascending=False)[:5]
# bottom_targets = pd.Series(target_f.sum()).sort_values()[:5]
# fig, axs = plt.subplots(figsize=(9,9) , nrows=2)
# sns.barplot(top_targets.values , top_targets.index , ax = axs[0] ).set(title = "Top five targets")
# sns.barplot(bottom_targets.values , bottom_targets.index, ax = axs[1] ).set(title = "bottom five targets")
# plt.show()

In [ ]:
# tail_enders = pd.Series(target_f.sum()).sort_values()[:70]

In [ ]:
# tail_enders = list(tail_enders.index)

In [ ]:
# tail_enders

In [ ]:
# merged_drug = folds.merge(train_drugs)
# df1.merge(df2, left_on='lkey', right_on='rkey')

In [ ]:
# merged_drug['drug_id'].value_counts()

In [ ]:
# lets get the drug_ids of interest tail_enders
# merge_grouped = merged_drug.groupby(by = 'drug_id')[target_cols].sum()


# targets with less representation




In [ ]:
# merge_grouped['sum_targets'] = merge_grouped[target_cols].sum(axis=1) 
# merge_grouped['sum_tail_enders'] = merge_grouped[tail_enders].sum(axis=1) 

In [ ]:
# drug_ids = list(merge_grouped[merge_grouped['sum_tail_enders'] >0].index)


In [ ]:

# x = merged_drug['drug_id'].value_counts()


In [ ]:
# max(x[drug_ids]) # totally expandable not the best way to do things but this idea was insane from start.

In [ ]:
# all_drugs =  merged_drug['drug_id'].value_counts()

In [ ]:
# samp_strat = {}

In [ ]:
# # get the sampling strat lets produce 150 of each will later come up with a better logic
# samp_strat = {}
# for i in list(all_drugs.index):
#     if i in drug_ids:
#         samp_strat.update({i:150})
#     else:
#         if((all_drugs[i]+1) < 7 ):
#             samp_strat.update({i:7})
#         else:
#             samp_strat.update({i:all_drugs[i]+1})
        
    
        
        
    





In [ ]:
# samp_strat # at this point its getting out of hand but ok its just for fun

In [ ]:
# drug_smote = pd.get_dummies(merged_drug, columns=['cp_time','cp_dose'])

In [ ]:
# y_smote = merged_drug['drug_id']

# x_smote = drug_smote.drop(target_cols+['sig_id' , 'kfold', 'drug_id'] , axis =1)

In [ ]:
# from imblearn.over_sampling import RandomOverSampler
# smote = RandomOverSampler(sampling_strategy= samp_strat ,  random_state=42)



In [ ]:
# X_res, y_res = smote.fit_resample(x_smote, y_smote) # i realized i over achieved but i was too deep to give up now so will see it to the end

In [ ]:
# merged_drug_to_targets = merged_drug.groupby(by = 'drug_id')[target_cols].max()

In [ ]:
# train_smote = pd.concat([X_res, y_res], axis=1)

In [ ]:
# merged_drug_to_targets

In [ ]:
#50,817 newly genrated_data
# train_smote_f= train_smote.merge(merged_drug_to_targets , on = 'drug_id')

In [ ]:
# train_smote_f =  train_smote_f.drop(['drug_id'], axis =1)

In [ ]:
# train_smote_f

In [ ]:
# target_f =  train_smote_f[target_cols]
# top_targets = pd.Series(target_f.sum()).sort_values(ascending=False)[:20]
# bottom_targets = pd.Series(target_f.sum()).sort_values()[:10]
# fig, axs = plt.subplots(figsize=(9,9) , nrows=2)
# sns.barplot(top_targets.values , top_targets.index , ax = axs[0] ).set(title = "Top five targets")
# sns.barplot(bottom_targets.values , bottom_targets.index, ax = axs[1] ).set(title = "bottom five targets")
# plt.show()
# # indeed beautiful achives the purpose but assumptions are too much


In [ ]:
# train_smote_f = train_smote_f.fillna(0)
# folds = train_smote_f.copy()

# mskf = MultilabelStratifiedKFold(n_splits=7)

# for f, (t_idx, v_idx) in enumerate(mskf.split(X=train_smote_f.drop(target_cols, axis=1), y= train_smote_f[target_cols])):
#     folds.loc[v_idx, 'kfold'] = int(f)

# folds['kfold'] = folds['kfold'].astype(int)
# folds

In [ ]:
# train = train_smote_f

## SAMPLING ENDS

### 4.6 Dataset Classes



In [ ]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct    

In [ ]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

### 4.7 Smoothing



In [ ]:
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

### 4.8 Preprocessing


In [ ]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [ ]:
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]
len(feature_cols)

## 5. Modeling



In [ ]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 7
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

num_features=len(feature_cols)
num_targets=len(target_cols)
num_non_scored_targets=len(target_non_scored_cols)
hidden_size=1500





In [ ]:
folds

In [ ]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(Dropout_Model)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(Dropout_Model)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x
    
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))    

In [ ]:
def run_training(fold, seed , pretrain):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    if pretrain:
        model = Model(
            num_features=num_features,
            num_targets=num_non_scored_targets, # non scored targets
            hidden_size=hidden_size,
        )
        
        # Load pretrained model
        model.load_state_dict(torch.load(f"FOLD{fold}_non_scored.pth"))
        
        # Reinitialize last layers
        model.batch_norm3 = nn.BatchNorm1d(hidden_size)
        model.dropout3 = nn.Dropout(0.5)
        model.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
        
    else:
        model = Model(
            num_features=num_features,
            num_targets=num_targets,
            hidden_size=hidden_size,
        )
    model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    
    if pretrain:
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-3, epochs=EPOCHS, steps_per_epoch=len(trainloader))
#         scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.5, patience=10, 
#                                                          threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
        epochs = 25
    else:
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
        epochs = copy.copy(EPOCHS)
    
    
    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
   
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold}_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    
    model.load_state_dict(torch.load(f"FOLD{fold}_.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions

In [ ]:
train_x = process_data(folds_non_scored)

In [ ]:
val_idx = train_x[train_x['kfold'] == 0].index

In [ ]:
val_idx.shape

In [ ]:
target_non_scored

In [ ]:
def run_training_non_scored(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds_non_scored)
    test_ = process_data(test)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_non_scored_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_non_scored_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_non_scored_targets,
        hidden_size=hidden_size,
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-4, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target_non_scored.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(10):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_fn, trainloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold}_non_scored.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_non_scored_targets,
        hidden_size=hidden_size,
    )
    
    model.load_state_dict(torch.load(f"FOLD{fold}_non_scored.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target_non_scored.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions

In [ ]:
def run_k_fold_none_scored(NFOLDS, seed):
    oof = np.zeros((len(train_non_scored), len(target_non_scored_cols)))
    predictions = np.zeros((len(test), len(target_non_scored_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training_non_scored(fold, seed)
        print(oof_.shape)
        print(oof.shape)
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

## 6. Prediction & Submission 



In [ ]:
def run_k_fold(NFOLDS, seed , pretrain=False):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed, pretrain)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [ ]:
SEED = [42, 1, 2, 3, 4, 5, 6]
# SEED = [0, 1]

oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

run_k_fold_none_scored(NFOLDS, SEED[0])



In [ ]:
for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed, pretrain=True)
    
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = predictions

In [ ]:
train

In [ ]:
# Averaging on multiple SEEDS


# oof = np.zeros((len(train), len(target_cols)))
# predictions = np.zeros((len(test), len(target_cols)))

# for seed in SEED:
    
#     oof_, predictions_ = run_k_fold(NFOLDS, seed)
#     oof += oof_ / len(SEED)
#     predictions += predictions_ / len(SEED)

# train[target_cols] = oof
# test[target_cols] = predictions


In [ ]:
train_targets_scored

In [ ]:
len(target_cols)

In [ ]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)    
#0.015

In [ ]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv('submission.csv', index=False)

In [ ]:
sub.shape

[Go to Top](#0)

In [ ]:
sub